In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error , r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures

### Processing csv data and separately storing dependent and independent variables

In [2]:
file = pd.read_csv(r"D:\IITM Acads\sem 8\ML for Engg\Assignment - 2\Concrete_Data.csv")
#file.head()
X = file.drop(file.columns[-1], axis = 1)
#x.insert(0, column = "Bias", value = 1)  
#x.drop(X.columns[0])
Y = file[file.columns[-1]]
#X_new = X.to_numpy()
#Y_new = Y.to_numpy()
#x.shape

### Using Normal equation to evaluate the weights 

Comparing the predictions obtained using normal equation and the one we used in previous assignment:

Since we cannot explicitely calculate the weights of the regression line in the previous work, we stick to comparing them on the basis of their mean square errors and R square fit for the predicted values.

1) Sklearn model:
    
    a) MSE for test and train data: 95.76159570981089 and 112.76375625798755 respectively
    b) R square for test: 0.6260931924425932
    
2) Normal equation model:

    a) MSE for test and train data: 95.29309432168549 and 112.69768869786847 respectively 
    b) R square for test: 0.6699690789892276

We see that the MSE values are almost similar in both the models, but a sginificant reduction in error value is observed in the normal equation model. This may be because of the relatively small data size (1030x9) and the matrix operations invloved in normal method is faster than the repeated calculations involved in the gradient decent method used in sklearn model.

In [12]:
X_train, X_test,  Y_train, Y_test = train_test_split(X, Y, test_size = 0.3)

pol = PolynomialFeatures(degree = 1, include_bias = True)
X_train_one = pol.fit_transform(X_train)
X_test_one = pol.transform(X_test)

weights = np.linalg.inv(X_train_one.T.dot(X_train_one)).dot(X_train_one.T).dot(Y_train)

Y_predicted_test = X_test_one.dot(weights)
Y_predicted_train = X_train_one.dot(weights)
print("Weights",weights)
print("Mean Squares Error (Test):", mean_squared_error(Y_test, Y_predicted_test))
print("Mean Squares Error (Train):", mean_squared_error(Y_train, Y_predicted_train))
print("R square (Test):", r2_score(Y_test, Y_predicted_test))

Weights [-2.50281202e+01  1.18100928e-01  1.03787877e-01  8.60664196e-02
 -1.56422557e-01  3.01822527e-01  1.79508774e-02  2.43782120e-02
  1.13463628e-01]
Mean Squares Error (Test): 103.36026116847432
Mean Squares Error (Train): 109.33880839709342
R square (Test): 0.6199490343470171


 ### Introducing L2 regularisation parameter to the model

The results after adding a regularisation term to the error equation has improved the quality of prediction. Depending on the value of hyper parameter $(\lambda)$ that we choose, the prediction can vary. 

Reasons for the increase in quality:

The regularisation term keeps a check on the problem if overfitting of model. Whenever the calculated weight goes beyond the minimal value, this regularisation term penalises the error function and the system approaches minimal at a slower rate thus properly predicting the weights.\
In our model, the differences in mean square error of predictions made without using L2 regularisation and with using it was found to be 5.318757217764869e-06 for a lambda value of 0.099. This shows with L2, the model is performing well.

Similarly for other values of lambda we can arrive at different error values.


In [11]:
I = np.identity(len(file.columns))
I[0,0] = 0
lamda = 0.099
theta_l2 = np.linalg.inv(X_train_one.T.dot(X_train_one) + lamda*I).dot(X_train_one.T).dot(Y_train)
Y_predicted_test_l2 = X_test_one.dot(theta_l2)
Y_predicted_train_l2 = X_train_one.dot(theta_l2)
#print(Y_predicted_test)
print("Mean Squares Error (Test):", mean_squared_error(Y_test, Y_predicted_test) - mean_squared_error(Y_test, Y_predicted_test_l2))
print("Mean Squares Error (Train):", mean_squared_error(Y_train, Y_predicted_train) - mean_squared_error(Y_train, Y_predicted_train_l2))
#print("R square (Test):", r2_score(Y_test, Y_predicted_test_l2) - r2_score(Y_test, Y_predicted_test))

Mean Squares Error (Test): 5.082056020455639e-07
Mean Squares Error (Train): -1.1652900866465643e-12


### Quadratic Model

a) The input for quadratic model now has 45 features

    
c) The results obtained using quadratic model is much better than the linear model. This can be observed in the significantly reduced value of mean squared errors for both the training and testing data: 49.97168114285114 and 61.707908332130638 respectively. Thus we can infer that using a polynomial function, we can have a better fit for our data set.


d) If we want a cubic model, we would require 165 features
 

In [15]:
pol = PolynomialFeatures(degree = 2, include_bias = True)
X_train_poly = pol.fit_transform(X_train)
X_test_poly = pol.transform(X_test)
#X.head()
#print(X_train_poly.shape)

new_weights = np.linalg.inv(X_train_poly.T.dot(X_train_poly)).dot(X_train_poly.T).dot(Y_train)

Y_predicted_test_poly = X_test_poly.dot(new_weights)
Y_predicted_train_poly = X_train_poly.dot(new_weights)

print("No.of input features:", X_train_poly.shape[1])
print("Weights",new_weights)
print("Mean Squares Error (Test):", mean_squared_error(Y_test, Y_predicted_test_poly))
print("Mean Squares Error (Train):", mean_squared_error(Y_train, Y_predicted_train_poly))

No.of input features: 45
Weights [-5.54283710e+03  3.53150031e+00  2.96407574e+00  3.25204804e+00
  1.51001381e+01  2.12939267e+01  3.54719860e+00  4.42522680e+00
  3.92612731e-02 -5.36173742e-04 -8.99044533e-04 -8.20552003e-04
 -5.43363895e-03 -8.30773185e-03 -9.86544363e-04 -1.31431431e-03
  2.21975188e-04 -4.38079220e-04 -5.23410602e-04 -4.19650711e-03
 -6.77981613e-03 -9.73769150e-04 -1.01830259e-03  4.26677206e-04
 -3.77522825e-04 -5.89569005e-03 -1.40954261e-02 -9.50168126e-04
 -1.06601284e-03  6.90644131e-04 -8.99064333e-03 -2.04026841e-02
 -5.21034004e-03 -6.00248503e-03 -1.11299649e-04 -3.44049503e-02
 -6.83499636e-03 -8.42285151e-03  3.94387494e-03 -5.77839198e-04
 -1.33537257e-03 -2.12819374e-05 -9.60525446e-04  2.48275251e-04
 -5.99822420e-04]
Mean Squares Error (Test): 55.01241456259077
Mean Squares Error (Train): 53.757795123945805
